### 상품 추천, 분석에는 여러 방법들이 있음 

- <b>연관규칙분석, 장바구니분석 (Association Rule Analysis, Market Basket Analysis)</b> : 고객의 대규모 거래데이터로부터 <u>함께 구매가 발생하는 규칙(예: A 동시에 B)을 도출하여, 고객이 특정 상품 구매 시 이와 연관성 높은 상품을 추천</u>
<br><br>
- <b>순차분석 (Sequence Analysis)</b> : 고객의 시간의 흐름에 따른 구매 패턴(A 일정 시간 후 B)을 도출하여, <u>고객이 특정 상품 구매 시 일정 시간 후 적시에 상품 추천</u>
<br><br>
- <b>Collaborative Filtering</b> : 모든 고객의 상품 구매 이력을 수치화하고, 추천 대상이 되는 고객A와 다른 고객B에 대해 상관계수를 비교해서, <u>서로 높은 상관이 인정되는 경우 고객B가 구입 완료한 상품 중에 고객A가 미구입한 상품을 고객A에게 추천</u>
<br><br>
- <b>Contents-based recommendation</b> : 고객이 <u>과거에 구매했던 상품들의 속성과 유사한 다른 상품 아이템 중 미구매 상품을 추천</u> (↔ Collaborative Filtering은 유사 고객을 찾는 것과 비교됨)
<br><br>
- <b>Who-Which modeling</b> : 특정 상품(군)을 추천하는 모형을 개발 (예: 신형 G5 핸드폰 추천 스코어모형)하여 구매 가능성 높은(예: 스코어 High) 고객(군) 대상 상품 추천

이중에서도 오늘의 TIL은 우선 연관규칙분석을 살펴볼 예정,, 예전에 많이 하기는 했는데, 실제 분석에 활용해서 액션까지 만든 케이스는 없었음, 이번에 복습하고 최근 분석 건에 적용해볼 예정

---

연관규칙분석은 이름에서 의미하는 그대로 규칙 베이스임, <br>

1 : (사과,치즈.생수) <br>
2: (생수,호두,치즈,고등어) <br>
3: (수박,사과,생수) <br>
4: (생수,호두,치즈,옥수수) <br>

이런 데이터셋이 있을 경우 이하와 같은 규칙들이 만들어 질 수 있음 

- 사과를 산 사람은 생수를 산다. 
- 호두를 산 사람은 치츠를 산다. 
- 호두를 산 사람은 생수를 산다. <br>
...
<br>
이런 많은 규칙 중에서 어떤 규칙이 좋은 규칙인지 알 수 있는 판단지표로써 3가지가 있다. 

1. 지지도(support) : <u>한 거래 항목 안에 A와 B를 동시에 포함하는 거래의 비율</u>. 지지도는 A와 B가 함께 등장할 확률이다. 전체 거래의 수를 A와 B가 동시에 포함된 거래수를 나눠주면 구할 수 있음   
2. 신뢰도(confidence) : <u>항목 A가 포함하는 거래에 A와 B가 같이 포함될 확률</u>, 신뢰도는 조건부 확률과 유사함. A가 일어났을 때 B의 확률이다. A의 확률을 A와 B가 동시에 포함될 확률을 나눠주면 구할 수 있음.
3. 향상도(lift) : <u>A가 주어지지 않을 때의 품목 B의 확률에 비해 A가 주어졌을 때 품목 B의 증가 비율임</u>. B의 확률이 A가 일어났을 때 B의 확률을 나눴을 때 구할 수 있음. lift값은 1이면 서로 독립적인 관계이며, 1보다 크면 두 품목이 서로 양의 상관관계, 1보다 작으면 두 품목이 서로 음의 상관관계로 볼 수 있다. A와 B가 독립이면 분모, 분자가 같기 때문이 1이 산출된다. 



--- 

### 연관규칙분석 알고리즘 
연관규칙분석의 대표적인 알고리즘은 3가지가 있음, 그 중 Apiori 알고리즘을 이하에서 실습했음, Apiori 알고리즘이 널리 쓰이는 이유는 알고리즘 구현이 비교적 간단하고 높은 수준의 성능을 보이기 때문임

<b style="color:blue">1. Apriori algorithm </b> <br>
2. FP-Growth algorithm <br>
3. DHP algorithm

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder 
from mlxtend.frequent_patterns import apriori, association_rules

mlxtend : 일상적인 데이터 사이언스 작업에 유용한 도구들로 구성된 파이썬 라이브러리라고 함.

In [4]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

fit함수로 dataset에 고유한 라벨을 가지게 함. transform함수를 통해서 파이썬 리스트를 one-hot 인코딩 된 numPy배열로 변환함

In [5]:
df

,Apple,Corn,Eggs,Ice cream,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,True,False,True,True,True,False,True
1,False,False,True,False,False,True,True,False,True
2,True,False,True,False,True,False,False,False,False
3,False,True,False,False,True,False,False,True,True
4,False,True,True,True,False,False,True,False,False


In [6]:
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

지지도 0.5 이상의 경우를 알아보면,

In [7]:
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,0.6,(Milk)
2,0.6,(Onion)
3,0.6,(Yogurt)
4,0.6,"(Onion, Eggs)"


즉, 5개의 장바구니 중에 Eggs가 4개 들어가 있으므로 0.8의 값이 도출됨.

In [8]:
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
1,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6


- association_rules 함수를 이용하여 지지도가 0.5가 넘는 항목에 대해 향상도가 양의 상관관계인 것을 확인하면 egg와 Onion이 양의 상관관계 인 것을 알 수 있음.<br> 
좀 더 해석하면, 
- 양파를 사면 무조건 달걀을 삼. (confidence = 1)
- lift지표는, A라는 상품에서 신뢰도가 동일한 상품 B와 C가 존재할 때, 어떤 상품을 더 추천해야 좋을지 판단할 때 사용.
- leverage는 Onion -> Eggs 케이스 때, 0.6 - 0.6*0.8로 0.6-0.48 = 0.12가 산출됩니다. 이는 Onion과 Eggs가 같이 등장하는 빈도와 Onion과 Eggs가 독립일 때 같이 등장할 기대빈도의 차이를 나타낸 것이다. 만약 Onion과 Eggs가 독립이면 Leverage는 0 임
- conviction은 (1-0.6)/(1-0.75) = 1.6 으로, 높은 Conviction값은 후반부 아이템이 전반부 아이템에 높은 의존을 보이고 있다는 것을 의미합니다. 

#### reference
- https://hezzong.tistory.com/entry/python-%EC%97%B0%EA%B4%80%EA%B7%9C%EC%B9%99%EB%B6%84%EC%84%9DA-Priori-Algorithm
- https://seunghuni96.tistory.com/112

#### TODO
- https://ordo.tistory.com/89
- https://ordo.tistory.com/93 <br>
실제 Apriori Algorithm 구현을 해보면 좋겠다! 